# <center>Projeto AirBNB</center>
  <center>Previsão de preços das acomodações em Boston/MA - USA</center>

#### 1. Entendendo o desafio:

##### 1.1. O que é o Airbnb?

O <a href="www.airbnb.com">Airbnb</a> é um serviço que permite que pessoas do mundo inteiro ofereçam suas casas para usuários que buscam acomodações mais em conta em qualquer lugar do mundo. No Airbnb, é possível oferecer um apenas um quarto ou a casa completa a outros usuários, como também alugar um espaço, caso seja a sua necessidade ou interesse. Usado por turistas, viajantes e profissionais em trânsito, o grande apelo do serviço está nos custos mais baixos e na facilidade de uso: alugar um imóvel sem muita burocracia.

Caso o usuário precise alugar um espaço, basta fazer uma busca (tanto no aplicativo como pelo site) para encontrar opções na cidade em que deseja. É possível escolher a partir de preços, observar as datas disponíveis (há casos de usuários que oferecem suas casas e cômodos por períodos curtos e específicos apenas), a partir das resenhas de outros usuários (você é classificado e pode classificar os proprietários) e assim por diante.
Para alugar uma acomodação, a mecânica é basicamente a mesma: é preciso cadastrar o imóvel disponível, especificando informações práticas a seu respeito (é a casa inteira ou só um quarto? O período é de um feriado específico, algumas semanas em que você vai viajar? O ano inteiro?), deve colocar fotos que mostrem o espaço e descrevê-lo da melhor maneira possível. Detalhes como endereço e localização também são muito importantes.

O Airbnb começou em 2008, quando dois designers que tinham um espaço sobrando hospedaram três viajantes que procuravam um lugar para ficar. Agora, milhões de anfitriões e viajantes optam por criar uma conta gratuita no Airbnb para que possam anunciar seu espaço e reservar acomodações únicas, em qualquer lugar do mundo. Além disso, os anfitriões de experiências do Airbnb compartilham suas paixões e interesses com viajantes e moradores locais.

##### 1.2. Objetivo:

Construir um modelo de previsão de preço que permita que o proprietário do imóvel (<i>host</i>) saiba quanto deve cobrar com base nas características do seu bem.

Ou ainda, que um locatário saiba se o preço que está sendo cobrado é justo para o imóvel escolhido.


#### 2. Extração/Obtenção de dados:

A base de dados utilizada nesse modelo pode ser baixado diretamente no site do <a href="www.kaggle.com">Kaggle</a>: <a href="https://www.kaggle.com/katerynaosadchuk/boston-airbnb-listings?select=boston_listings.csv">Clique Aqui para baixar!</a> 

###### 2.1. Importando a biblioteca e a base de dados:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
df = pd.read_csv('boston_listings.csv')
display(df)

#### 3. Ajuste/Limpeza de dados:

Aqui vamos analisar quais as informações que são importantes manter no dataframe, excluindo tudo que não demonstrar ser necesário.

Para analisar melhor, vamos criar uma planilha em excel com os 300 primeiros registros para poder fazer a análise das informações.

In [ ]:
print(list(df.columns)) # Esse código nos mostra todas as colunas do dataframe, porém não acessamos o conteúdo.
                        # Daí a necessidade de criar o excel com os primeiros registros

In [ ]:
# Criando os Excel com os primeiros 300 registros. O arquivo fica salvo na mesma pasta do notebook.
df.head(300).to_excel('primeiros300.xlsx')

Após a análise, temos alguns tipos de colunas que iremos excluir:
- as que não tenham informações relevantes para o modelo de predição, por ex: ids e links;
- as que tenham informações parecidas com a de outras colunas;
- as colunas que sejam de texto livre preenchidas pelo *host*;
- colunas em que todas ou quase todas as informações são iguais;

Com isso, ficamos com as seguintes colunas:

'host_response_time',	'host_response_rate',	'host_is_superhost',	'host_total_listings_count',	'latitude',	'longitude',	'property_type',	'room_type',	'accommodates',	'bathrooms',	'bedrooms',	'beds',	'bed_type',	'amenities_dict',	'price',	'cleaning_fee',	'number_of_reviews',	'review_scores_rating',	'review_scores_accuracy',	'review_scores_cleanliness',	'review_scores_checkin',	'review_scores_communication',	'review_scores_location',	'review_scores_value',	'instant_bookable',	'is_business_travel_ready',	'cancellation_policy'

###### 3.1. Criando o dataframe apenas com as colunas necessárias:

In [ ]:
colunas = ['host_response_time', 'host_response_rate', 'host_is_superhost', 'host_total_listings_count', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities_dict', 'price', 'cleaning_fee', 'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'is_business_travel_ready', 'cancellation_policy']

#Aqui criaremos um filtro para o dataframe aparecer só as colunas que colocamos em uma lista na variável acima

df = df.loc[:, colunas]
display(df)

O dataset original tinha 51 colunas, com a análise exploratória ficamos com 27.

###### 3.2. Tratando as informações vazias (NaN):

Temos que fazer essa análise pois as informações vazias podem atrapalhar o modelo de previsão. Analisando abaixo quais as colunas que contem muita falta de informação para podermos excluí-las.

In [ ]:
print(df.isnull().sum())

Temos algumas colunas com mais de 10% de informações vazias, com isso iremos excluir todas elas da base de dados.

In [ ]:
#excluindo as colunas com muitos dados vazios.

for coluna in df:
    if df[coluna].isnull().sum() >= 350:
        df = df.drop(coluna, axis=1)

        
display(df)
print(df.shape)

In [ ]:
print(df.isnull().sum())

Com esse novo tratamento, nos restaram **17 colunas** na base de dados que irá para o modelo de previsão.

Para os outros dados, como o número de NaNs é pequeno, iremos excluir as linhas que tenham falta das informações.

In [ ]:
display(df)
df = df.dropna()
print(df.shape)
#print(df.isnull().sum())

Agora temos um dataframe com **3820 linhas** e as mesmas **17 colunas**.

###### 3.3. Verificando o tipo de dados:

Aqui vamos verificar como o pandas reconheceu o tipo do dado, pois, algumas vezes ele reconhece um número (int) como um texto (string) e aí atrapalha o processamento.

Após a análise, iremos corrigir eventuais erros.

In [ ]:
print(df.dtypes)
#abaixo vemos que o preço, que deveria ser um float, retornou como objeto em virtude do simbolo $
#para corrigir teremos que excluir o $ e a virgula que consta como separador de milhar

In [ ]:
df['price'] = df['price'].str.replace('$', '') #retira o $
df['price'] = df['price'].str.replace(',', '') #retira a ,
df['price'] = df['price'].astype(np.float32, copy=False) #transforma em float
print(df.dtypes) #para confirmar que foi efetuada a mudança


#### 4. Análise exploratória:

Vamos percorrer todas as colunas que sobraram e analisar os valores que contem ali dentro. Qual é o maior e o menor valor, em que faixa estão a maioria dos valores e etc.

Aqui teremos que usar alguns conceitos de estatística que serão explicados a medida que forem sendo usados.

Primeiro, vamos ver a correlação entre os itens, após utilizaremos o conceito estatistico de quartil.

O objetivo final da análise exploratória é descobrir alguma informação muito discrepante (outliers), para depois decidir se iremos manter ou não esses outliers.

Tais análises irão iniciar com as colunas númericas como o preço, depois iremos para quartos, camas e hóspedes. Por fim, iremos analisar as colunas com texto para definir quais fazem sentido em continuar na análise ou não.

###### 4.1. Correlação de informações:

É como um item se correlaciona com o outro, por exemplo, quando se tem mais quartos o preço é maior. Nesse caso, seria uma correlação positiva.

Ou quando aumento a quantidade mínima de diárias o preço da diária abaixa, aí seria uma correlação negativa.



In [ ]:
print(df.corr())

In [ ]:
#demonstrando a mesma informação com um mapa de calor
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot=True, cmap='Reds')


Verificamos aqui que não há nenhuma correlação tão forte que possa atrapalhar nosso modelo de previsão, logo deixaremos todas as colunas.

###### 4.2. Quartil:



Após ver a correlação, vamos usar o conceito de **quartil**, ou seja, vamos pegar nossos dados e dividi-los em quatro quartis.

Para excluir os **outliers** (dados muito discrepantes), vamos excluir os que estejam abaixo do Q1 - 1,5x Amplitude e acima do Q3 + 1,5x Amplitude.

Amplitude é a diferença entre o Q1 e o Q3.

Vamos definir algumas funções para ajudar na análise de **outliers** das colunas.



In [ ]:
def limites(coluna):
    q1 = coluna.quantile(0.25)
    q3 = coluna.quantile(0.75)
    amplitude = q3 - q1
    return q1 - 1.5 * amplitude, q3 + 1.5 * amplitude

def excluir_outliers(df, nome_coluna):
    linhas = df.shape[0]
    lim_inf, lim_sup = limites(df[nome_coluna])
    df = df.loc[(df[nome_coluna] >= lim_inf) & (df[nome_coluna] <= lim_sup), :] #filtrando o dataframe
    linhas_removidas = linhas - df.shape[0]
    return df, linhas_removidas
    

In [ ]:
def diagrama_caixa(coluna):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(15,5)
    sns.boxplot(x=coluna, ax=ax1)
    ax2.set_xlim(limites(coluna))
    sns.boxplot(x=coluna, ax=ax2)
    
def histograma(coluna):
    plt.figure(figsize=(15,5))
    sns.distplot(coluna, hist=True)

###### 4.2.1. Analisando a coluna de preços (price):

In [ ]:
diagrama_caixa(df['price'])
histograma(df['price'])

O **gráfico superior esquerdo**, mostra todos os preços de diárias, com isso, podemos ver que temos preços até próximo a US$ 10000 (dez mil dólares) por dia.

Já o **gráfico superior direito** mostra a variação de preços dentro dos limites definidos anteriormente. Logo, verificamos que ficou os preços entre 0 a pouco mais de 300 dólares a diária. 

Levando isso em consideração, temos que tomar a decisão se usaremos todos os dados ou se iremos excluir os **outliers**. Para tomar essa decisão, precisamos de uma análise qualitativa.

Como nosso objetivo é verificar preço de diárias de imóveis "comuns" nós provavelmente iremos excluir os outliers, pois, um imóvel de cerca de 10 mil dólares a diária deve ser uma mega mansão super luxuosa e que só irá atrapalhar o modelo de predição.

Eu disse provavelmente por que teremos que ver a quantidade de dados que serao excluídos, pois se forem um número muito alto, talvez não possamos excluir, já que isso vai indicar que realmente temos muitas unidades com valores acima dos nossos limites.

In [ ]:
df, linhas_removidas = excluir_outliers(df, 'price') 
print(f'{linhas_removidas} linhas removidas')



Como a quantidade de linhas removidas não foi tão substancial, vamos deixar elas excluidas para não atrapalhar nosso modelo.

Vamos ver como fica nosso gráfico de histograma após a exclusão dos outliers da coluna price.

In [ ]:
histograma(df['price'])

In [ ]:
print(df.shape)
# ainda temos 3596 linhas.

Agora vamos continuar analisando as outras colunas para saber se poderemos excluir os outliers ou não.

###### 4.2.2. Analisando a coluna de host_total_listings_count:

In [ ]:
diagrama_caixa(df['host_total_listings_count'])
histograma(df['host_total_listings_count'])

Neste caso, vemos no histograma que a quase totalidade possui um imóvel listado, então, em vez de usar um histograma, vamos criar um gráfico de barras que vai dizer quantas imóveis as pessoas tem. 

In [ ]:
def grafico_barra(coluna):
    plt.figure(figsize=(15,5))
    ax = sns.barplot(x=coluna.value_counts().index, y=coluna.value_counts())
    ax.set_xlim(limites(coluna))


In [ ]:
#grafico_barra(df['host_total_listings_count'])
plt.figure(figsize=(15,5))
sns.barplot(x=df['host_total_listings_count'].value_counts().index, y=df['host_total_listings_count'].value_counts())

In [ ]:
df, linhas_removidas = excluir_outliers(df, 'host_total_listings_count') 
print(f'{linhas_removidas} linhas removidas')

In [ ]:
print(df.shape)

###### 4.2.3. Analisando a coluna de accommodates:

In [ ]:
diagrama_caixa(df['accommodates'])
grafico_barra(df['accommodates'])

In [ ]:
df, linhas_removidas = excluir_outliers(df, 'accommodates') 
print(f'{linhas_removidas} linhas removidas')
print(df.shape)

###### 4.2.4. Analisando a coluna de bathrooms:

In [ ]:
diagrama_caixa(df['bathrooms'])
plt.figure(figsize=(15,5))
sns.barplot(x=df['bathrooms'].value_counts().index, y=df['bathrooms'].value_counts())

In [ ]:
#df, linhas_removidas = excluir_outliers(df, 'bathrooms') 
#print(f'{linhas_removidas} linhas removidas')
print(df.shape)

Como o limite que o cálculo de quartil chegou para banheiros foi de apenas 1. Acho melhor não excluir, tendo em vista que imóveis com mais banheiros devem sim modificar o preço, bem como é bem comum um imóvel ter mais banheiros. Por isso, como as linhas já foram excluídas, irei rodar novamente o código, deixando a parte da exclusão como comentário.

###### 4.2.5. Analisando a coluna de bedrooms:

In [ ]:
diagrama_caixa(df['bedrooms'])
plt.figure(figsize=(15,5))
sns.barplot(x=df['bedrooms'].value_counts().index, y=df['bedrooms'].value_counts())

Pelo mesmo motivo que em banheiros, como os limites definidos para quartos foi de apenas um. Também não excluirei os outliers

###### 4.2.6. Analisando a coluna de beds:

In [ ]:
diagrama_caixa(df['beds'])
grafico_barra(df['beds'])

In [ ]:
df, linhas_removidas = excluir_outliers(df, 'beds') 
print(f'{linhas_removidas} linhas removidas')
print(df.shape)

###### 4.2.7. Analisando a coluna de quantidade de reviews:

In [ ]:
diagrama_caixa(df['number_of_reviews'])
grafico_barra(df['number_of_reviews'])

Como o modelo vai prever o valor de um novo imóvel, é melhor tirar essa coluna, pois quem for incluir seu imóvel não vai ter nenhum review no início.

In [ ]:
df = df.drop('number_of_reviews', axis=1)
print(df.shape)

Com isso, finalizamos nossa análise exploratória das colunas numéricas. Terminamos com 2661 linhas e 16 colunas.

Passaremos agora para as colunas de texto. Esse tipo de coluna geralmente é uma categoria, por exemplo, se é casas ou apartamentos.

###### 4.3. Analisando as colunas de texto:

As features de texto serão separados pelas colunas de verdadeiro e falso (true / false) e os que tem algumas categorias de texto.

Os verdadeiro e falso não precisa muito tratamento, as outras sim.

###### 4.3.1. Tipo de propriedade (property_type):

Primeiro, teremos que ver a quantidade de que cada texto (categoria) está na tabela. Para isso iremos mostrar através de um print.

In [ ]:
print(df['property_type'].value_counts())

plt.figure(figsize=(15,5))
grafico = sns.countplot('property_type', data=df)
grafico.tick_params(axis='x', rotation=90)

Vamos mudar algumas categorias que apresentam menos resultados, agrupando tudo em uma coluna que se chamará Outros.

No caso, escolhemos manter todas as categorias de cima para baixo até o Loft. A categoria Hotel já fará parte da nova categoria Outros.

In [ ]:
tipo_imovel = df['property_type'].value_counts()

agrupar = []
for tipo in tipo_imovel.index:
    if tipo_imovel[tipo] < 30:
        agrupar.append(tipo)
#print(agrupar)

for tipo in agrupar:
    df.loc[df['property_type'] == tipo, 'property_type'] = 'Outros'
    
print(df['property_type'].value_counts())
plt.figure(figsize=(15,5))
grafico = sns.countplot('property_type', data=df)
grafico.tick_params(axis='x', rotation=90)
        

###### 4.3.2. Tipo de sala (room_type):

Iremos fazer praticamente a mesma análise do item anterior. Iremos verificar os tipos e agrupar caso esteja muito pulverizado algum tipo de informação.

In [ ]:
print(df['room_type'].value_counts())

plt.figure(figsize=(15,5))
grafico = sns.countplot('room_type', data=df)
grafico.tick_params(axis='x', rotation=90)

Neste caso, como são poucas categorias (4 no total) e ainda duas com um quantidade de repetição muito baixa, entendemos que isso não atrapalhará o modelo, logo deixaremos essa coluna inalterada.

###### 4.3.3. Tipo de cama (bed_type):

In [ ]:
print(df['bed_type'].value_counts())

plt.figure(figsize=(15,5))
grafico = sns.countplot('bed_type', data=df)
grafico.tick_params(axis='x', rotation=90)

Aqui temos o mesmo caso que aconteceu com o room_type. Então vamos deixar as categorias inalteradas.

###### 4.3.4. Política de cancelamento (cancellation_policy):

In [ ]:
print(df['cancellation_policy'].value_counts())

plt.figure(figsize=(15,5))
grafico = sns.countplot('cancellation_policy', data=df)
grafico.tick_params(axis='x', rotation=90)

Aqui, como temos duas categorias bem parecidas, vamos juntá-las e criar uma categoria chamada super_strict.

In [ ]:
tipo_cancelamento = df['cancellation_policy'].value_counts()

agrupar = []
for tipo in tipo_cancelamento.index:
    if tipo_cancelamento[tipo] < 30:
        agrupar.append(tipo)
#print(agrupar)

for tipo in agrupar:
    df.loc[df['cancellation_policy'] == tipo, 'cancellation_policy'] = 'super_strict'
    
print(df['cancellation_policy'].value_counts())
plt.figure(figsize=(15,5))
grafico = sns.countplot('cancellation_policy', data=df)
grafico.tick_params(axis='x', rotation=90)

###### 4.3.5. Amenities (amenities_dict):

In [ ]:
print(df['amenities_dict'].iloc[1])

print(type(df['amenities_dict'].iloc[1]))

Como podemos ver, apesar de parecer um dicionário, no nosso banco de dados as informações dos amenities são uma string. 

Então, primeiramente vamos ter que transformar em um dicionário, para depois saber quantos amenities tem no imóvel e colocar esse número de amenities em uma coluna separada.

A razão disso, é que não queremos analisar item por item de cada apartamento e sim quantos itens cada apartamento tem, ou seja, o parametro será a quantidade de amenities e não cada amenitie individualmente.

In [ ]:
print(df['amenities_dict'][0])

In [ ]:
import ast

for i, amenidades in enumerate(df['amenities_dict']):
    df['amenities_dict'][i] = ast.literal_eval(amenidades)


In [ ]:
print(type(df['amenities_dict'].iloc[0]))

Agora que já mudamos todos as informações para um dicionário, vamos conseguir contar quantas amenidades cada imóvel possui.

In [ ]:
df = df.assign(Amenindades=0)

for i in range(len(df['amenities_dict'])):
    count = 0
    try:
        for j in df['amenities_dict'][i].values():
            if j > 0:
                count += 1                
        print(count)
        df['Amenindades'][i] = count
    except:
        pass


In [ ]:
print(df.columns)
#df = df.drop(df[2661:].index)
print(df['Amenindades'])

In [ ]:
df = df.drop(df[df.Amenindades == 0].index)
print(df['Amenindades'])

Agora que foi criada a coluna amenidades, com o número de itens que cada uma possuia, podemos excluir a coluna amenities_dict.

In [ ]:
df = df.drop('amenities_dict', axis=1)
print(df.shape)

A coluna Amenidades agora é uma coluna numérica, então teremos que fazer a análise dela para ver se tem outliers para ser excluída ou não.

###### 4.3.5.1. Analisando a coluna Amenidades:

In [ ]:
diagrama_caixa(df['Amenindades'])
grafico_barra(df['Amenindades'])

Verifica-se facilmente que a grande maioria possui 23 amenidades e que os limites definidos ficaram entre 10 e 40, agora vamos excluir os outliers.

In [ ]:
df, linhas_removidas = excluir_outliers(df, 'Amenindades') 
print(f'{linhas_removidas} linhas removidas')
print(df.shape)

###### 5. Visualização de mapa das propriedades:

Iremos usar a biblioteca plotly, pois ela cria um gráfico interativo.



In [ ]:
print(df.columns)

In [ ]:
import plotly.express as px


amostra = df.sample(n=1700)
centro_mapa = {'lat': amostra.latitude.mean(), 'lon': amostra.longitude.mean()}
mapa = px.density_mapbox(amostra, lat='latitude', lon='longitude', z='price', radius=6, center=centro_mapa, zoom=11, mapbox_style='open-street-map')
mapa.show()

#### 5. Encoding:

O encoding irá facilitar o treino do modelo de previsão, já que tais modelos só funcionam com variáveis numéricas.

Ou seja, tudo que for texto, teremos que transformar em números.

###### 5.1. Transformando as colunas de Verdadeiro (True) e Falso (False):

Aqui, iremos transformar as colunas host_is_superhost, instant_bookable e is_business_travel_ready que tem como resposta verdadeiro em falso em respostas da seguinte forma:
1 - Verdadeiro (True)
0 - Falso (False)

In [ ]:
print(df.iloc[0])

In [ ]:
#criando uma coluna com as categorias de verdadeiro e falso
colunas_tf = ['host_is_superhost', 'instant_bookable', 'is_business_travel_ready' ]

df_encoded = df.copy()

for coluna in colunas_tf:
    df_encoded.loc[df_encoded[coluna] == 't', coluna] = 1
    df_encoded.loc[df_encoded[coluna] == 'f', coluna] = 0
    #               ^       linha filtrada  ˆ 

print(df_encoded.iloc[0])

No dataframe df_encoded já podemos ver que o t e f viraram 1 e 0, respectivamente.

###### 5.2. Transformando as colunas com diversas categorias:

Nesse caso, não podemos simplesmente colocar os números, pois o modelo pode entender que há uma relação de ordens, ou que o item 4 é o dobro do item 2, sendo que no texto são apenas categorias diferentes que não tem nenhum correlação.

Por isso, iremos criar as **variáveis Dummies**. Na prática, substituiremos a informação da categoria por uma nova coluna (nova categoria) que terá a informação se aquela informação que estava inserida na categoria é verdadeira ou falsa.

Por exemplo, em vez da informação Real_bed na categoria Beds, teremos uma categoria Real_Bed que dirá se a informação é verdadeira ou falsa (1 ou 0).

In [ ]:
coluna_categorias = ['property_type', 'room_type','bed_type', 'cancellation_policy']

df_encoded = pd.get_dummies(data=df_encoded, columns=coluna_categorias)
display(df_encoded.head())

In [ ]:
print(df_encoded.columns)

Aqui, finalizamos o tratamento dos dados.

Agora partiremos para o treinamento do modelo.

#### 6. Teoria para o Modelo de Previsão:

É aqui que entra o Machine Learning.

Vamos seguir alguns passos para criar e treinar o modelo.

Passo 1: definir se é Classificação ou Regressão; <br>
Passo 2: Escolher as métricas para avaliar o modelo;<br>
Passo 3: Escolher quais modelos vamos usar/testar;<br>
Passo 4: Treinar os modelos e testar;<br>
Passo 5: Comparar os resukltados dos modelos e escolher o melhor;<br>
Passo 6: Analisar o melhor modelos mais a fundo;<br>
Passo 7: Fazer os ajustes no melhor modelo<br>

###### 6.1. Classificação ou Regressão:

**Classificação**: É quando o modelo de machine learning vai separar em categorias, por exemplo: um diagnóstico, definir se aquele email é um SPAM, etc.

**Regressão**: Nesse caso, o modelo vai chegar a um número específico, por exemplo: o preço de determinado item, a velocidade, etc.

Com isso, fica claro que *in casu* temos um problema de **regressão**.

###### 6.2. Métricas para avaliar o modelo:

O melhor modelo é o que erra menos ou o que acerta mais?

Iremos usar duas métricas para avaliar o melhor modelo:

**R²**: 
Vai de 0 a 1 -> Quanto maior melhor.
Mede "o quanto" dos valores o modelo consegue explicar.
Exemplo: 92% significa que o modelo consegue explicar 92% da variância dos dados a partir das informações que entregamos a ele.

**RSME (Erro quadrático médio ou Raiz do Erro Quadrático Médio)**:
Pode ser qualquer valor.
Quanto menor o valor, será melhor o modelo.
Mede o quanto o modelo erra.

###### 6.3. Escolha dos modelos que vamos usar/testar:

- **Linear Regression (Regressão Linear)**: é um método estatistico que usa a relação entre os dados para "desenhar" uma linha reta entre eles. Essa linha poderá ser usada para prever valores futuros;
- **Random Forest Regressor**: é um modelo de árvore de decisão, onde cada pergunta vai separando os valores em diversas categorias até chegar na resposta mais provável possível. No Random Forest, ele vai pegar todos os dados e criar várias árvores de decisão com pedaços menores de dados e depois vai fazer uma média com as respostas prováveis;
- **Extra Trees**: também é um modelo de árvore de decisão e faz a mesma coisa que o Random Forest, com a diferença que no Extra Trees, ele faz uma pergunta aleatória, já no Random Forest ele faz a melhor pergunta possível.



###### 6.4. Treinar e testar os modelos:

Primeiro temos que separar os dados aleatoriamente em 2 conjutos:
- Treino
- Teste

O treino são os dados que o modelo irá usar para aprender. <br>
O teste são os dados que usamos para ver se o modelo aprendeu bem.

Exemplo: 80% de dados para o treinoe 20% para o teste.

Sempre avaliamos o resultado nos testes para não correr o risco de overfitting.


###### 6.5. Comparar os Resultados do Teste e escolher o vencedor:

Iremos calcular o R² e RSME para cada modelo e escolheremos uma métrica para ser a principal.

A outra métrica será considerado critério de desempate ou para comparar modelos com resultados principais parecidos.





###### 6.6. Analisar o melhor modelo mais a fundo:

Após escolher o vencedor, vamos analisá-lo mais a fundo buscando ver como ele funciona e identificar a importancia de cada feature. Com isso, buscamos oportunidades de melhoria do próprio modelo vencedor.

Caso uma feature (coluna) não seja utilizada ou usada muito pouco, podemos fazer um teste retirando ela da base de dados e ver se o resultado melhora ou não, com base nas métricas:

- R2 ou RSME;
- Velocidade do modelo;
- Simplicidade do modelo

###### 6.7. Fazer ajustes no melhor modelo:

Testamos cada mudança que fizermos no modelo para fazer um ajuste final.

Analisamos se as features identificadas podem ser retiradas e treinamos e testamos o modelo, sempre comparando com o resultado original e o resultado anterior.

Tudo isso com o objetivo de:
- Encontrar uma possível melhoria no modelo;
- Ver se conseguimos chegar no mesmo resultado ou mais próximo com um modelo mais simples e/ou mais rápido;
- Fazer outros tipos de testes que tenham sido planejados (por exemplo, não retirar algum outlier ou coluna já excluida)



#### 7. Modelo de Previsão na prática:

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split


- Métricas de Avaliação:

In [ ]:
# y_teste serão os preços reais
# previsao será o valor que foi previsto para podermos comparar 
# com o valor real

def avaliar_modelo(nome_modelo, y_teste, previsao):
    r2 = r2_score(y_teste, previsao)
    rsme = np.sqrt(mean_squared_error(y_teste, previsao))
    return f'Modelo {nome_modelo}:\nR2: {r2}\nRSME: {rsme}'

Eixo X são todas as variáveis. <br>
Eixo Y é o que queremos prever.


- Modelos a serem testados:

    1. RandomForest
    2. LinearRegression
    3. Extra Tree

In [ ]:
modelo_rf = RandomForestRegressor()
modelo_lr = LinearRegression()
modelo_et = ExtraTreesRegressor()

modelos = {'RandomForest': modelo_rf,
          'LinearRegression': modelo_lr,
          'ExtraTrees': modelo_et
          }

y = df_encoded['price']
X = df_encoded.drop('price', axis=1)

- Separar os dados em treino e teste:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

for nome_modelo, modelo in modelos.items():
    #treinar
    modelo.fit(X_train, y_train)
    #testar
    previsao = modelo.predict(X_test)
    print(avaliar_modelo(nome_modelo, y_test, previsao))

- Analisando o modelo em busca de melhoridas:

In [ ]:
importancia = pd.DataFrame(modelo_rf.feature_importances_, X_train.columns)
importancia = importancia.sort_values(by=0, ascending=False)
display(importancia)

In [ ]:
plt.figure(figsize=(15,5))
ax = sns.barplot(x=importancia.index, y=importancia[0])
ax.tick_params(axis='x', rotation=90)

- Retirando coluna para tentar melhorar o modelo:

is_business_travel_ready não teve muito impacto no modelo, então vamos excluir essa feature e testar o modelo sem ela.

In [ ]:
df_encoded = df_encoded.drop('is_business_travel_ready', axis=1) 

y = df_encoded['price']
X = df_encoded.drop('price', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

for nome_modelo, modelo in modelos.items():
    #treinar
    modelo.fit(X_train, y_train)
    #testar
    previsao = modelo.predict(X_test)
    print(avaliar_modelo(nome_modelo, y_test, previsao))

In [ ]:
print(X_test.columns)

In [ ]:
importancia = pd.DataFrame(modelo_rf.feature_importances_, X_train.columns)
importancia = importancia.sort_values(by=0, ascending=False)
display(importancia)

In [ ]:
print(len(importancia))

dicionario = {'property_type_Apartment': 0, 'property_type_Bed and breakfast': 0, 'property_type_Condominium': 0, 'property_type_Guest suiteHouse': 0, 'property_type_Loft': 0, 'property_type_Townhouse': 0, 'property_type_Outros': 0, 'room_type_Entire home/apt': 0, 'room_type_Hotel room': 0, 'room_type_Private room': 0, 'room_type_Shared room': 0, 'bed_type_Airbed': 0, 'bed_type_Couch': 0, 'bed_type_Futon': 0, 'bed_type_Pull-out Sofa': 0, 'bed_type_Real Bed': 0, 'cancellation_policy_flexible': 0, 'cancellation_policy_moderate': 0, 'cancellation_policy_strict_14_with_grace_period': 0, 'cancellation_policy_super_strict': 0}

print(len(dicionario))

O resultado não mudou quase nada, mas como ficou mais simples (com uma coluna a menos) vamos manter assim.

Fiz o teste também tirando o bed_type, mas aí o resultado deu uma leve piora.

Considero essa acurácia de 64% um bom valor, levando em consideração o tamanho do banco de dados utilizado, já que, com pouco mais de 2 mil linhas, era um dataset relativamente pequeno.

#### 8. Deploy do projeto/modelo:

É colocar em produção. É deixar o seu modelo disponível para outros usuários.

1. Criar arquivo do modelo (joblib)
2. Fazer o deploy com o streamlit
3. Criar um novo arquivo python
4. Importar o streamlit e criar o código
5. Atribuir ao botão o carregamento do modelo
6. Deploy finalizado
    

###### 8.1. Criar arquivo joblib:

Pegamos o valor X, que são as características do imóvel. Então vamos salvar essa base de dados.


In [ ]:
X['price'] = y #estamos recolocando a coluna de preço na base de dados X

X.to_csv('dados.csv')

In [ ]:
import joblib

joblib.dump(modelo_rf, 'modelo.joblib')

Com esse arquivo, não precisa treinar o modelo toda a vez que for fazer uma previsão. Iremos puxar esse arquivo que criamos quando for prever outro imóvel.

##### 8.2 / 8.3. Criar um novo arquivo e fazer o deploy com streamlit:

Como teremos que criar o novo arquivo, será nesse novo arquivo que continuaremos com a finalização deste pr